## Daily SLIE GeoTIFF Curation

* Prescribe new file names
* Reprojecting these data to 3338 on an aligned grid, and override the source compression and lack of `nodata` value.

In [ ]:
from pathlib import Path

import dask
import dask.distributed as dd
from dask_jobqueue import SLURMCluster

import eda as eda
import preprocess as preprocess
from luts import ice_zones, ice_years, data_sources
from config import INPUT_DIR, DAILY_BEAUFORT_DIR, DAILY_CHUKCHI_DIR, SCRATCH_DIR

In [ ]:
ice_zones

In [ ]:
beauf_geotiffs = []
chuk_geotiffs = []

for region in ice_zones:
    for year in ice_years:
        target_dir = Path(INPUT_DIR / region / year)
        tiffs = eda.list_geotiffs(target_dir, str_to_match="dailyslie")
        if region == "Beau":
            beauf_geotiffs = beauf_geotiffs + tiffs
        elif region == "Chuk":
            chuk_geotiffs = chuk_geotiffs + tiffs
        else:
            print(f"{target_dir} is not expected.")

In [ ]:
# example of the renaming function, mostly just placing the product in the output directory and then also expanding the data source attribute
print(preprocess.daily_slie_rename(beauf_geotiffs[0]))
print(preprocess.daily_slie_rename(chuk_geotiffs[-1]))

In [ ]:
cluster = SLURMCluster(
            cores=24,
            memory="128GB",
            queue="t2small",
            walltime="23:00:00",
            log_directory=SCRATCH_DIR,
            local_directory=SCRATCH_DIR,
            account="cmip6",
            interface="ib0",
        )
client = dd.Client(cluster)

cluster.scale(100)

In [ ]:
tasks = [dask.delayed(preprocess.tap_reproject_daily_slie_raster)(f) for f in beauf_geotiffs]
future = dask.compute(*tasks)

In [ ]:
tasks = [dask.delayed(preprocess.tap_reproject_daily_slie_raster)(f) for f in chuk_geotiffs]
future = dask.compute(*tasks)

In [ ]:
client.close()

In [ ]:
cluster.scale(0)

In [ ]:
cluster.close()

In [ ]:
assert len(eda.list_geotiffs(DAILY_BEAUFORT_DIR)) == len(beauf_geotiffs)

In [ ]:
assert len(eda.list_geotiffs(DAILY_CHUKCHI_DIR)) == len(chuk_geotiffs)